### Analyses of output files from the pipeline optimus v6.0.0+
#### Aim of this part of analysis is predict the cell cycle

#### Load necessary libraries and useful functions

In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import scanpy as sc

In [3]:
import warnings
warnings.filterwarnings("ignore") 

#### Data Loading

In [5]:
input_h5ad = "../scAtlas/tmp/10k_pbmc_v3_out_filtered_cellbenderDefault_lowQcell_scDblFinder.h5ad"
adata = sc.read_h5ad(input_h5ad)

In [6]:
display(adata)

AnnData object with n_obs × n_vars = 11018 × 58347
    obs: 'background_fraction', 'cell_probability', 'cell_size', 'droplet_efficiency', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'outlier', 'scDblFinder_score', 'scDblFinder_class'
    var: 'ambient_expression', 'feature_type', 'genome', 'gene_id', 'cellbender_analyzed', 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: 'barcode_indices_for_latents', 'barcodes_analyzed', 'barcodes_analyzed_inds', 'cell_size_lognormal_std', 'empty_droplet_size_lognormal_loc', 'empty_droplet_size_lognormal_scale', 'estimator', 'features_analyzed_inds', 'fraction_data_used_for_testing', 'learning_curve_learning_rate_epoch', 'learning_curve_learning_rate_value', 'learning_curve_test_elbo', 'learning_curve_test_epoch', 'learning_curve_train_e

###### check barcode and gene names

In [7]:
display(adata.var_names)
display(adata.obs_names)

Index(['DDX11L1', 'WASH7P', 'MIR6859-1', 'MIR1302-2HG', 'MIR1302-2', 'FAM138A',
       'AL627309.6', 'OR4G11P', 'OR4F5', 'AL627309.1',
       ...
       'pRNA-11', 'RNA5-8S5', 'pRNA-12', 'RNA5-8SN2', 'AC007325.3',
       'AC007325.1', 'AC007325.4', 'AC007325.2', 'U6-36', 'U1-15'],
      dtype='object', name='gene_name', length=58347)

Index(['CAGAGCCTCTTCGATT', 'TACATTCTCCGTAGTA', 'CACTAAGGTCACCACG',
       'ATCACGAAGGGACACT', 'TGTTACTCATAACTCG', 'ATAGACCGTCAGGCAA',
       'AATCGACGTTTCGTAG', 'ATCTTCATCCCGAATA', 'TGTTACTTCTACAGGT',
       'AGACCCGAGAATACAC',
       ...
       'CTTTCGGAGATCGCTT', 'CCCTCTCCACAGCTTA', 'CCACAAAAGCGTCTCG',
       'TTCTGTACAGCAGTTT', 'CACCGTTCAATCCTAG', 'CCTAAGACAGCGAACA',
       'CTCCTTTGTACAGTTC', 'CCTACGTAGCACCTGC', 'GAGGGATAGTAAACGT',
       'TATTGGGGTCATACCA'],
      dtype='object', name='barcode', length=11018)

#### Cell cycle prediction

In [ ]:
! pip install biomart

In [ ]:
cell_cycle_genes = pd.read_csv(
    "https://raw.githubusercontent.com/hbc/tinyatlas/master/cell_cycle/Homo_sapiens.csv"
)  # This is the same source as the automated Seurat function
cell_cycle_genes

sc.tl.score_genes_cell_cycle(
    adata_cellbender,
    s_genes=cell_cycle_genes[cell_cycle_genes.phase == "S"].geneID.unique(),
    g2m_genes=cell_cycle_genes[cell_cycle_genes.phase == "G2/M"].geneID.unique(),
)
#TODO better for filtered cell or all cell

In [ ]:
adata.layers["log_transformed"] = np.log1p(adata.X)
adata.to_df(layer="log_transformed")
